In [ ]:
import sys
import os
os.environ['OPP4SAR_DIR'] = '/home/jhewers/Documents/meng_project/code/'
sys.path.insert(0,os.getenv('OPP4SAR_DIR'))
DIR = os.path.join(os.getenv('OPP4SAR_DIR'),'generated_data/prob_map_jackton')

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({
    'axes.grid':False,
    'figure.dpi':300,
    'image.cmap':'gray',
    'legend.loc':'upper left',
    'legend.fancybox':True,
    })
def legend_decorator(method):
    def decorate_legend(*args,**kwargs):
        cust_kwargs = {'bbox_to_anchor':(1.05, 1)}
        cust_kwargs.update(kwargs)
        return method(*args,**cust_kwargs)
    return decorate_legend
plt.legend = legend_decorator(plt.legend)

import json
import numpy as np
import scipy.stats as st
import scipy as sp
import os
import csv
from PIL import ImageColor, Image

import utm

from src.json_helpers import GlobalJsonDecoder
from src.data_models.positional.waypoint import Waypoint, Waypoints
from src.data_models.probability_map import ProbabilityMap
from src.waypoint_generation import WaypointFactory, WaypointAlgSettings
from src.enums import WaypointAlgorithmEnum
from src.simulation.vehicle import VehicleSimData

In [ ]:
from __future__ import division
import math
MERCATOR_RANGE = 256

def  bound(value, opt_min, opt_max):
  if (opt_min != None): 
    value = max(value, opt_min)
  if (opt_max != None): 
    value = min(value, opt_max)
  return value

def  degreesToRadians(deg) :
  return deg * (math.pi / 180)

def  radiansToDegrees(rad) :
  return rad / (math.pi / 180)

class G_Point:
    def __init__(self,x=0, y=0):
        self.x = x
        self.y = y

class G_LatLng:
    def __init__(self,lt, ln):
        self.lat = lt
        self.lng = ln

class MercatorProjection:
    def __init__(self) :
      self.pixelOrigin_ =  G_Point( MERCATOR_RANGE / 2, MERCATOR_RANGE / 2)
      self.pixelsPerLonDegree_ = MERCATOR_RANGE / 360
      self.pixelsPerLonRadian_ = MERCATOR_RANGE / (2 * math.pi)

    def fromLatLngToPoint(self, latLng, opt_point=None) :
      point = opt_point if opt_point is not None else G_Point(0,0)
      origin = self.pixelOrigin_
      point.x = origin.x + latLng.lng * self.pixelsPerLonDegree_
      # NOTE(appleton): Truncating to 0.9999 effectively limits latitude to
      # 89.189.  This is about a third of a tile past the edge of the world tile.
      siny = bound(math.sin(degreesToRadians(latLng.lat)), -0.9999, 0.9999)
      point.y = origin.y + 0.5 * math.log((1 + siny) / (1 - siny)) * -     self.pixelsPerLonRadian_
      return point

    def fromPointToLatLng(self,point) :
          origin = self.pixelOrigin_
          lng = (point.x - origin.x) / self.pixelsPerLonDegree_
          latRadians = (point.y - origin.y) / -self.pixelsPerLonRadian_
          lat = radiansToDegrees(2 * math.atan(math.exp(latRadians)) - math.pi / 2)
          return G_LatLng(lat, lng)
    
    #pixelCoordinate = worldCoordinate * pow(2,zoomLevel)
    
def getCorners(center, zoom, mapWidth, mapHeight):
    scale = 2**zoom
    proj = MercatorProjection()
    centerPx = proj.fromLatLngToPoint(center)
    SWPoint = G_Point(centerPx.x-(mapWidth/2)/scale, centerPx.y+(mapHeight/2)/scale)
    SWLatLon = proj.fromPointToLatLng(SWPoint)
    NEPoint = G_Point(centerPx.x+(mapWidth/2)/scale, centerPx.y-(mapHeight/2)/scale)
    NELatLon = proj.fromPointToLatLng(NEPoint)
    return {
        'N' : NELatLon.lat,
        'E' : NELatLon.lng,
        'S' : SWLatLon.lat,
        'W' : SWLatLon.lng,
    }

In [ ]:
from src.waypoint_generation import CostFunc
cost_func = CostFunc()

wp_gen_settings = WaypointAlgSettings.Global()

In [ ]:
with open(os.path.join(DIR,"output_sar_ps.json"),'r') as f:
    data_sar = json.load(f,cls=GlobalJsonDecoder)
    print(type(data_sar))
with open(os.path.join(DIR,"output_sim_ps.json"), 'r') as f:
    data_sim = json.load(f,cls=GlobalJsonDecoder)
    print(type(data_sim))
with open(os.path.join(DIR,"output_wp_ps.json"), 'r') as f:
    data_wp = json.load(f,cls=GlobalJsonDecoder)
    print(type(data_wp))

R = 15 

In [ ]:
import AirDataUAV as ADU 
from PIL import Image

d1 = ADU.UAVData("/home/jhewers/Documents/meng_project/ps_data/2020-11-19_14-18-29.csv")
d2 = ADU.UAVData("/home/jhewers/Documents/meng_project/ps_data/2020-11-19_14-39-52.csv")
d3 = ADU.UAVData("/home/jhewers/Documents/meng_project/ps_data/2020-11-19_15-16-33.csv")

img = ProbabilityMap.fromPNG("/home/jhewers/Documents/meng_project/code/img/probability_imgs/prob_map_8_jackton.png")
img_location = ProbabilityMap.fromPNG("/home/jhewers/Documents/meng_project/code/img/probability_imgs/prob_map_8_jackton_map.png")

center = G_LatLng(55.751849, -4.238594)
corners = getCorners(center, 17, 1280/2,1280/2)
N,W,_,_ = utm.from_latlon(corners['N'],corners['W'])
S,E,_,_ = utm.from_latlon(corners['S'],corners['E'])

w_m,h_m = (W-E,S-N)
ORIGIN = np.array([N,W])

img = img.resampled(int(h_m),int(w_m))
img_location = img_location.resampled(int(h_m),int(w_m))

for d in [d1,d2,d3]:
    d.utm[0] -= ORIGIN[0]
    d.utm[1] = -(d.utm[1] - ORIGIN[1])

plt.imshow(img_location.toIMG())
plt.imshow(img.toIMG(),alpha = 0.4)

for dat in data_sim.data:
    x = dat[1].pos.x
    y = [f for f in dat[1].pos.y]
    plt.plot(x,y,label=str(dat[0]).split('.')[1].lower())
    plt.scatter(x[0],y[0])

plt.plot(d1.utm[0],d1.utm[1],label="PS ASU Inspector")
plt.scatter(d1.utm[0][0],d1.utm[1][0])
plt.plot(d2.utm[0],d2.utm[1],label="PS AUS Steven Preece")
plt.scatter(d2.utm[0][0],d2.utm[1][0])
plt.plot(d3.utm[0],d3.utm[1],label='Attempted Parallel Swaths')
plt.scatter(d3.utm[0][0],d3.utm[1][0])
plt.ylabel("y (m)")
plt.xlabel("x (m)")
plt.legend()
fig=plt.gcf()
fig.savefig(os.path.join(DIR,'paths_ps.png'))
plt.show()


In [ ]:
npts = 25
for d,label in zip([d1,d2,d3],["PS ASU Inspector","PS AUS Steven Preece","Attempted Parallel Swaths"]):
    file_path = os.path.join(DIR,f"prob_accum_w_t_{'_'.join(label.split())}.csv")
    t_arr= []
    c_arr = []
    if os.path.isfile(file_path):
        print(f"File {file_path} found")
        with open(file_path,'r') as f:
            csvreader = csv.DictReader(f) 
            for row in csvreader:
                c_arr.append(float(row['cost']))
                t_arr.append(float(row['time']))
    else:
        print(f"File {file_path} NOT found")
        wp_arr = []
        x = d.utm[0]
        y = d.utm[1]
        print(f"{label} - {npts} points to calculate...")
        for i in range(0,len(x),len(x)//npts):
            xi = x[i]
            yi = y[i]
            ti = d.t_s[i]
            wp_arr.append(Waypoint(xi,yi))
            wps = Waypoints(wp_arr)
            cost = cost_func.calculate(wps,img,R)
            t_arr.append(ti)
            c_arr.append(cost)    
            print(f"{label}: {i}/{len(x)}")        
        with open(file_path,'w') as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(['time','cost'])
            csvwriter.writerows([(f,g) for f,g in zip(t_arr,c_arr)])
    
    plt.plot(t_arr,c_arr, label=label)
    print(f"Final cost for {label} is {c_arr[-1]:.4f}")

alg,vehicle = data_sim.data[0]
file_path = os.path.join(DIR,f"prob_accum_w_t_{str(alg).split('.')[1].lower()}.csv")
t_arr= []
c_arr = []
if os.path.isfile(file_path):
    print(f"File {file_path} found")
    with open(file_path,'r') as f:
        csvreader = csv.DictReader(f) 
        for row in csvreader:
            c_arr.append(float(row['cost']))
            t_arr.append(float(row['time']))
else:
    wp_arr = []
    x = np.array(vehicle.pos.x)
    y = np.array(vehicle.pos.y)
    t = vehicle.t
    print(f"{alg} - {npts} points to calculate...")
    for i in range(0,len(x),len(x)//npts) :
        xi = x[i]
        yi = y[i]
        ti = t[i]
        wp_arr.append(Waypoint(xi,yi))
        wps = Waypoints(wp_arr)
        cost = cost_func.calculate(wps,img,R)
        t_arr.append(ti)
        c_arr.append(cost)    
        print(f"{alg} data: {i}/{len(x)}")
    with open(file_path,'w') as f:
            csvwriter = csv.writer(f)
            csvwriter.writerow(['time','cost'])
            csvwriter.writerows([(f,g) for f,g in zip(t_arr,c_arr)])
print(f"Final cost for {alg} is {c_arr[-1]:.4f}")
plt.plot(t_arr,c_arr, label=str(alg).split('.')[1])

plt.xlabel("Time (s)")
plt.ylabel("Accumulated probability")
plt.ylim([0, -1])
plt.legend()
fig = plt.gcf()
fig.savefig(os.path.join(DIR,"accumulated_probability_ps.png"))
plt.show()

In [ ]:
# PLOT SIMULATION
cutoff = 20*60

def plot(alg,d,img):
    fig, ax = plt.subplots()
    
    ax.imshow(img)
    wps = d.pos
    t = np.array(d.t)
    wps_x = np.array(wps.x)
    wps_y = np.array(wps.y)
    
    ax.plot(wps_x[np.where(t>=cutoff)],wps_y[np.where(t>=cutoff)],linewidth=3,label='Over 20mins')
    below = ax.plot(wps_x[np.where(t<cutoff)],wps_y[np.where(t<cutoff)],linewidth=3,label='Below 20mins')
    ax.scatter(wps_x[0],wps_y[0],50,color=below[0].get_color(),label=f'Start',zorder=100)


    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")

    mn,mx = ax.get_xlim()
    ax.set_xlim([mn-20,mx+20])
    mn,mx = ax.get_ylim()
    ax.set_ylim([mn+20,mx-20])
    
    s = str(alg).split('.')[1]
    fig.savefig(os.path.join(DIR,s+"_sim.png"))
    ax.set_title(s)
       
    return fig,ax

for key in data_sim.data:
    alg, output = key
    fig,ax = plot(alg,output,img)
    plt.tight_layout()
    plt.legend()
    plt.show()
    break

In [ ]:
# SAR SIM for PS data
import diskcache as dc
import tempfile

objs = np.array(data_sar.data)

for d,label in zip([d1,d2,d3],["PS ASU Inspector","PS AUS Steven Preece","Attempted Parallel Swaths"]):
    wps = data_wp.data[str(alg)]['wps']
    
    print(label)
    vehicle = VehicleSimData()
    cache = dc.Cache(os.path.join(tempfile.gettempdir(),'opp4sar',str(np.random.rand())[2:]))
    
    max_x,max_y = np.max(objs,axis=0)
    min_x,min_y = np.min(objs,axis=0)
    x,y = np.meshgrid(np.arange(min_x,max_x+1),np.arange(min_y,max_y+1))
    x,y=x.flatten(),y.flatten()
    objs_possible_xy = np.vstack((x,y)).T

    for xi,yi,ti in zip(d.utm[0],d.utm[1],d.t_s):
        # create distance vectors
        v = np.array([xi-objs_possible_xy[:,0],
                    yi-objs_possible_xy[:, 1]])

        # if distance is < search radius -> object found
        dist = np.linalg.norm(v, axis=0)

        if any(dist < R):
            inds = np.where(dist < R)[0]
            for ind in inds:
                loc = objs_possible_xy[ind]
                cache[f'{loc[0]},{loc[1]}'] = ti                            
            objs_possible_xy = np.delete(objs_possible_xy, inds, axis=0)

    for key in cache.iterkeys():
        t = cache[key]
        xy=tuple(np.array(key.split(',')).astype(float).astype(int))
        inds = np.where((objs[:,0]==xy[0])&(objs[:,1]==xy[1]))[0]
        for ind in inds:
            vehicle.found.append(
                (t,objs[ind])
            )

    data_sim.data.append((label,vehicle))
    del vehicle, cache

In [ ]:
total_objects = len(data_sar.data)
colors = np.array([ImageColor.getcolor(f, "RGB") for f in 
         ['#b71c1c',
          '#311b92',
          '#ff6f00', 
          '#4fc3f7',
          '#d500f9',
          '#1b5e20']])/255

t_hists = {}
for alg,vehicle in data_sim.data:
    v_found = np.array(vehicle.found)[:,0]
    if alg in t_hists:
        t_hists[alg] = np.append(t_hists[alg],v_found)
    else:
        t_hists[alg] = v_found

dct = {}
cutoff = 20*60

for i,(alg,t_hist) in enumerate(t_hists.items()):

    t_hist = t_hist.astype(float)
    t_hist = t_hist[np.where(t_hist<cutoff+5*60)]
    print(f"{alg}: {100*len(t_hist)/total_objects:.2f}% found ({len(t_hist)} out of {total_objects})")
    mean = np.mean(t_hist)
    
    if '.' in (alg:=str(alg)):
        alg = alg.split('.')[1].lower()

    dct[str(alg)] = {
        'color':colors[i],
        't_hist_s':t_hist,
        't_hist_min':t_hist/60,
        'mean':np.mean(t_hist),
        'before_endurance_limit':np.sum(t_hist<cutoff)/total_objects
        }
    print(f"{alg} w/ mean time to found = {mean:.2f}s and found before endurance limit ({cutoff}s) = {100*np.sum(t_hist<cutoff)/total_objects:.2f}%")

values = [f['t_hist_min'] for f in dct.values()]
plt.hist(values,bins = 10,label=dct.keys())

plt.ylabel("Found count at time")
plt.xlabel("Time till object found (minutes)")

plt.axvspan(20,30,edgecolor='w',facecolor='r',alpha=0.3,zorder=-100,label='Over endurance',hatch='///')


plt.xlim([0,np.max([np.max(f) for f in values])])
plt.tight_layout()
plt.legend()
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"time_till_object_found_ps.png"))  



In [ ]:
def plot_bar(x,y,x_label,y_label):

    fig,ax = plt.subplots(constrained_layout=True)
    plt.xticks(rotation=45)
    ax.set_ylabel(y_label)
    ax.set_xlabel(x_label)
    ax.grid(True, linestyle='--', which='major',
                       color='grey', alpha=.25)
    rects = ax.bar(x,y)   

    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

    max_y = np.max(y)
    ax.set_ylim([0, int(np.ceil(max_y / 100.0)) * 110])
    
    return fig,ax

plot_bar([f for f in dct],[100*dct[f]['before_endurance_limit'] for f in dct],'','% found within endurance limit')
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"found_within_endurance.png"))

In [ ]:
x_max = -np.inf
for alg,d in dct.items():
    t_hist = d['t_hist_s']
    y = []
    x = []
    if max(t_hist)>x_max:x_max = max(t_hist)
    for i in np.linspace(0,max(t_hist)):
        x.append(i)
        y.append(100*np.sum(t_hist<i)/total_objects)

    plt.plot(x,y,label=f"{alg}",color=d['color'])

plt.legend(loc='lower right')
plt.xlim([0,x_max])
plt.ylabel("Percentage found (%)")
plt.xlabel("Endurance limited time-to-find (s)")
fig = plt.gcf()
plt.show()
fig.savefig(os.path.join(DIR,"endurance_limited_time-to-find_ps.png"))
